<a href="https://colab.research.google.com/github/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0009_tensor_rt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning in Medical AI 2018/2019 using PyTorch & Google Collab.


<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/drive/1JEIeD_445sFvcjSrITB5Z_oW8VHRS_kA">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/deeponcology/PyTorchMedicalAI/blob/master/shlomo_dl_0001_cuda_collab_pytorch.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

<img src="https://github.com/deeponcology/PyTorchMedicalAI/raw/master/assets/tumor_visdom.jpg" align="center" width=30%>

### Author: 
***Shlomo Kashani***, Head of AI at www.DeepOncology.AI, shlomo@deeponcology.ai 


### Synopsys:
Using TensorRT to extract VGG features using the paper:https://github.com/alexander-rakhlin/ICIAR2018 

<img src="https://github.com/alexander-rakhlin/ICIAR2018/raw/master/pics/nn_diagram.png" align="center" width=50%>

### DataSets:
https://github.com/alexander-rakhlin/ICIAR2018


# Collab notebook: 009 using TensorRT with Keras and TF

In [0]:
%reset -f

In [2]:
! wget https://gist.githubusercontent.com/ashokpant/5c4e9481615f54af4025ab2085f85869/raw/e09f10a30ce1b40018f4440df9de788f5d6fdad0/cuda_9.0_cudnn_7.0.sh
! chmod +x cuda_9.0_cudnn_7.0.sh
! sudo bash cuda_9.0_cudnn_7.0.sh

--2019-02-02 17:38:33--  https://gist.githubusercontent.com/ashokpant/5c4e9481615f54af4025ab2085f85869/raw/e09f10a30ce1b40018f4440df9de788f5d6fdad0/cuda_9.0_cudnn_7.0.sh
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1317 (1.3K) [text/plain]
Saving to: ‘cuda_9.0_cudnn_7.0.sh’

cuda_9.0_cudnn_7.0. 100%[===================>]   1.29K  --.-KB/s    in 0s      

2019-02-02 17:38:34 (214 MB/s) - ‘cuda_9.0_cudnn_7.0.sh’ saved [1317/1317]

--2019-02-02 17:38:39--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awai

In [8]:
! sudo apt-get --fix-broken install
! nvcc --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libcudnn7 libcudnn7-dev
The following packages will be upgraded:
  libcudnn7 libcudnn7-dev
2 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Need to get 285 MB of archives.
After this operation, 96.3 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  libcudnn7-dev 7.4.2.24-1+cuda10.0 [136 MB]
Get:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  libcudnn7 7.4.2.24-1+cuda10.0 [149 MB]
Fetched 285 MB in 2min 14s (2,130 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Re

# TRT version
We need to download this file and access it from Drive
- https://developer.nvidia.com/nvidia-tensorrt-4x-download
- https://developer.nvidia.com/compute/machine-learning/tensorrt/4.0/ga/TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
! pwd
! ls -la '/content/drive/My Drive/TRT/'

# trt_path='/content/drive/My Drive/TRT/TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1.tar.gz'


/content
total 798507
drwx------ 2 root root      4096 Feb  2 17:33 TensorRT-4.0.1.6
-rw------- 1 root root 410872039 Feb  2 14:04 TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.0.cudnn7.1.tar.gz
-rw------- 1 root root 406793742 Feb  2 12:37 TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.2.cudnn7.1.tar.gz


# References:
- https://docs.nvidia.com/deeplearning/sdk/tensorrt-install-guide/index.html
- https://tsmatz.wordpress.com/2018/07/07/tensorrt-tensorflow-python-on-azure-tutorial/
- https://jkjung-avt.github.io/tf-trt-models/
- https://kezunlin.me/post/dacc4196/
- https://blog.csdn.net/MacwinWin/article/details/80177326


# Install TRT form the TAR file
- https://kezunlin.me/post/dacc4196/

In [11]:
import os
os.chdir('/content/drive/My Drive/TRT/')
! tar zxvf 'TensorRT-4.0.1.6.Ubuntu-16.04.4.x86_64-gnu.cuda-9.0.cudnn7.1.tar.gz'

TensorRT-4.0.1.6/
TensorRT-4.0.1.6/data/
TensorRT-4.0.1.6/data/mnist/
TensorRT-4.0.1.6/data/mnist/5.pgm
TensorRT-4.0.1.6/data/mnist/mnist.caffemodel
TensorRT-4.0.1.6/data/mnist/deploy.prototxt
TensorRT-4.0.1.6/data/mnist/mnist_lenet.caffemodel
TensorRT-4.0.1.6/data/mnist/lenet5_custom_pool.uff
TensorRT-4.0.1.6/data/mnist/4.pgm
TensorRT-4.0.1.6/data/mnist/batches/
TensorRT-4.0.1.6/data/mnist/batches/batch851
TensorRT-4.0.1.6/data/mnist/batches/batch690
TensorRT-4.0.1.6/data/mnist/batches/batch396
TensorRT-4.0.1.6/data/mnist/batches/batch321
TensorRT-4.0.1.6/data/mnist/batches/batch964
TensorRT-4.0.1.6/data/mnist/batches/batch640
TensorRT-4.0.1.6/data/mnist/batches/batch254
TensorRT-4.0.1.6/data/mnist/batches/batch925
TensorRT-4.0.1.6/data/mnist/batches/batch756
TensorRT-4.0.1.6/data/mnist/batches/batch186
TensorRT-4.0.1.6/data/mnist/batches/batch928
TensorRT-4.0.1.6/data/mnist/batches/batch884
TensorRT-4.0.1.6/data/mnist/batches/batch980
TensorRT-4.0.1.6/data/mnist/batches/batch759
Tens

In [0]:
! sudo rm -rf /opt/TensorRT-4.0.1.6/
! sudo mv TensorRT-4.0.1.6 /opt/

In [13]:
! ls -la '/opt/TensorRT-4.0.1.6/'

total 1564
drwx------ 9 root root    4096 Jun 12  2018 .
drwxr-xr-x 1 root root    4096 Feb  2 18:03 ..
drwx------ 8 root root    4096 Jun 12  2018 data
drwx------ 5 root root    4096 Jun 12  2018 doc
drwx------ 2 root root    4096 Jun 12  2018 graphsurgeon
drwx------ 2 root root    4096 Jun 12  2018 include
drwx------ 4 root root    4096 Jun 12  2018 python
drwx------ 3 root root    4096 Jun 12  2018 targets
-r-------- 1 root root 1563257 Jun 12  2018 TensorRT-Release-Notes.pdf
drwx------ 2 root root    4096 Jun 12  2018 uff


In [0]:
import os 
os.chdir('/opt/TensorRT-4.0.1.6/')
! sudo ln -s TensorRT-4.0.1.6/ tensorrt


# TRT Python 

In [15]:
os.chdir('/opt/TensorRT-4.0.1.6/python/')
! ls -la 
!sudo pip3 install tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl 

total 3096
drwx------ 4 root root    4096 Jun 12  2018 .
drwx------ 9 root root    4096 Feb  2 18:03 ..
drwx------ 3 root root    4096 Jun 12  2018 data
drwx------ 4 root root    4096 Jun 12  2018 doc
-rw------- 1 root root 1571581 Jun 12  2018 tensorrt-4.0.1.6-cp27-cp27mu-linux_x86_64.whl
-rw------- 1 root root 1579848 Jun 12  2018 tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl
tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl is not a supported wheel on this platform.


In [0]:
! mv tensorrt-4.0.1.6-cp35-cp35m-linux_x86_64.whl tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl

In [17]:
!sudo pip3 install tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl 

Processing ./tensorrt-4.0.1.6-cp36-cp36m-linux_x86_64.whl


In [18]:
os.chdir('/opt/TensorRT-4.0.1.6/uff')
! sudo pip3 install uff-0.4.0-py2.py3-none-any.whl 


Processing ./uff-0.4.0-py2.py3-none-any.whl


In [19]:
!which convert-to-uff

/usr/local/bin/convert-to-uff


In [20]:
! ls -la /opt/TensorRT-4.0.1.6/graphsurgeon/
os.chdir('/opt/TensorRT-4.0.1.6/graphsurgeon')
! sudo pip3 install graphsurgeon-0.2.0-py2.py3-none-any.whl

total 24
drwx------ 2 root root  4096 Jun 12  2018 .
drwx------ 9 root root  4096 Feb  2 18:03 ..
-rw------- 1 root root 13338 Jun 12  2018 graphsurgeon-0.2.0-py2.py3-none-any.whl
Processing ./graphsurgeon-0.2.0-py2.py3-none-any.whl


In [21]:
! ls -la /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/
! ls -la /usr/local/cuda


total 20
drwx------  5 root root 4096 Jun 12  2018 .
drwx------  3 root root 4096 Jun 12  2018 ..
drwx------  2 root root 4096 Jun 12  2018 bin
drwx------  2 root root 4096 Jun 12  2018 lib
drwx------ 17 root root 4096 Jun 12  2018 samples
lrwxrwxrwx 1 root root 8 Feb  2 17:41 /usr/local/cuda -> cuda-9.0


In [22]:
ls -la /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/

total 20
drwx------  5 root root 4096 Jun 12  2018 ./
drwx------  3 root root 4096 Jun 12  2018 ../
drwx------  2 root root 4096 Jun 12  2018 bin/
drwx------  2 root root 4096 Jun 12  2018 lib/
drwx------ 17 root root 4096 Jun 12  2018 samples/


In [0]:
os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/" # NOTE NO + SIGN 
os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['LD_LIBRARY_PATH']+= ":/usr/lib64-nvidia/:/opt/TensorRT-4.0.1.6/include/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/"
os.environ['PATH']+= ":/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/"


In [24]:

# !env | grep cuda
!env | grep LD_LIBRARY_PATH
# /opt/TensorRT-4.0.1.6/include/

LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/lib64-nvidia/:/opt/TensorRT-4.0.1.6/include/:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib:/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin/


In [25]:
# ! ls -la  /opt/TensorRT-4.0.1.6/include
%%file /etc/ld.so.conf.d/tensorrt

#/opt/TensorRT-4.0.1.6/lib
/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib


Writing /etc/ld.so.conf.d/tensorrt


In [26]:
! sudo ldconfig

! sudo cp /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib* /usr/lib
! sudo cp /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin* /usr/bin
! sudo cp /opt/TensorRT-4.0.1.6/include/* /usr/local/include/

cp: -r not specified; omitting directory '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib'
cp: -r not specified; omitting directory '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/bin'


In [0]:
os.chdir('/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/samples/sampleMNIST')

In [0]:
#! make -j 8
# Makefile  sampleMNIST.cpp

#os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/" # NOTE NO + SIGN 
#os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
#os.environ['LD_LIBRARY_PATH']= "/opt/TensorRT-4.0.1.6/include/"



In [31]:
# see https://stackoverflow.com/questions/50621536/tensorrt-python-package-incompatibility-with-python-3-6/50793199 
! sudo find / -name *nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so

/usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so


In [0]:
! sudo mv /usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-35m-x86_64-linux-gnu.so /usr/local/lib/python3.6/dist-packages/tensorrt/infer/_nv_infer_bindings.cpython-36m-x86_64-linux-gnu.so

In [74]:
! wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb
! wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb  

--2019-02-02 18:35:41--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.232.112, 2606:2800:247:2063:46e:21d:825:102e
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.232.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122730426 (117M) [application/x-deb]
Saving to: ‘libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb.1’

libcudnn7_7.2.1.38- 100%[===================>] 117.04M   156MB/s    in 0.7s    

2019-02-02 18:35:41 (156 MB/s) - ‘libcudnn7_7.2.1.38-1+cuda9.0_amd64.deb.1’ saved [122730426/122730426]

--2019-02-02 18:35:43--  https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.232.112, 2606:2800:247:2063:46e:21d:825:102

In [0]:
# ! sudo find / -name tensorflow*



In [109]:
#! sudo dpkg -i libcudnn7_7.3.0.29-1+cuda9.0_amd64.deb
! sudo rm -rf /usr/local/lib/python3.6/dist-packages/tensorflow*
! sudo rm -rf /usr/local/lib/python3.6/dist-packages/keras*
! sudo pip3 uninstall tensorflow tensorflow-gpu protobuf keras
! sudo pip3 install tensorflow-gpu keras
! sudo pip3 install --upgrade tensorflow-gpu



Skipping tensorflow as it is not installed.
Skipping tensorflow-gpu as it is not installed.
Uninstalling protobuf-3.6.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/google/protobuf/*
    /usr/local/lib/python3.6/dist-packages/protobuf-3.6.1-py3.6-nspkg.pth
    /usr/local/lib/python3.6/dist-packages/protobuf-3.6.1.dist-info/*
Proceed (y/n)? Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/base_command.py", line 176, in main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/uninstall.py", line 75, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_install.py", line 823, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_uninstall.py", line 263, in remove
    if auto_confirm 

In [110]:
#! sudo find / -name libnvinfer.so.* 
! sudo ln -s /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4.1.2 /opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4
#/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/

os.environ['LD_LIBRARY_PATH']+= "/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/"

! python -c "import tensorflow.contrib.tensorrt as trt; print(trt.__path__)"


from tensorflow.contrib import tensorrt as trt
import tensorflow 
print (tensorflow.__version__)

ln: failed to create symbolic link '/opt/TensorRT-4.0.1.6/targets/x86_64-linux-gnu/lib/libnvinfer.so.4': File exists
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 24, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/__init__.py", line 88, in <module>
    from tensorflow.python import keras
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/__init__.py", line 25, in <module>
    from tensorflow.python.keras import applications
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/applications/__init__.py", line 22, in <module>
    import keras_applications
ModuleNotFoundError: No module named 'keras_applications'
1.12.0


# VGG in Keras

In [0]:
import os
import zipfile
import hashlib
from six.moves.urllib.error import URLError
from six.moves.urllib.request import urlretrieve
import argparse
from tqdm import tqdm
from keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Concatenate
import numpy as np

import numpy as np
import pandas as pd
import time, os, glob
import cv2

from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD, Adam
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input

from os.path import isfile, join, abspath, exists, isdir, expanduser
from os import listdir, makedirs, getcwd, remove

## Define the VGG feature extractor

In [0]:
from tqdm import tqdm # Enable progress bar
from keras.models import Model
from keras.preprocessing import image
from keras.layers import Flatten, Input

from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, warnings
    
class VGGFV:
    __name__ = "VGGFV"

    def __init__(self, feature_layer="block5_conv3", input_size=224,num_class=1,batch_size=32):
        base_model = VGG16(weights='imagenet', include_top=False,
                           input_shape=[input_size,input_size,3], classes=num_class)
                                        
        x2 = GlobalAveragePooling2D()(base_model.get_layer("block2_conv2").output)  # 128
        x3 = GlobalAveragePooling2D()(base_model.get_layer("block3_conv3").output)  # 256
        x4 = GlobalAveragePooling2D()(base_model.get_layer("block4_conv3").output)  # 512
        x5 = GlobalAveragePooling2D()(base_model.get_layer("block5_conv3").output)  # 512        
        x = Concatenate(name="concat_1408")([x2,x3,x4,x5])  
#         x= concatenate([x2,x3,x4,x5], name="concat_1408")

        model = Model(inputs=base_model.input, outputs=x)        
#         optimizer = Adam(lr=0.0001)
#         model.compile(loss='categorical_crossentropy',
#                       optimizer=optimizer,
#                       metrics=['accuracy'])
        
        self.model=model    
        self.batch_size = batch_size
        self.data_format = K.image_data_format()
    
    def predict(self, x):
        if self.data_format == "channels_first":
            x = x.transpose(0, 3, 1, 2)
        x = preprocess_vgg(x.astype(K.floatx()))
        return self.model.predict(x, batch_size=self.batch_size).squeeze()
    

In [108]:
img_path = "/content/drive/My Drive/TRT/image227.png"
feature_layer = "block5_conv3" # 512
input_size=224

img = image.load_img(img_path, target_size=(input_size, input_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_vgg(x)
print (x.shape)

model = VGGFV (feature_layer, input_size=input_size,num_class=1)
model.model.summary()


vgg_feature_vector = model.predict(x)
print (vgg_feature_vector.shape)
vgg_feature_vector

(1, 224, 224, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

UnknownError: ignored

In [0]:
# Do we have cuda?!
!which nvcc
!nvcc --version

/usr/local/cuda/bin/nvcc
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148


total 16
drwxr-xr-x 1 root root 4096 Jan 31 17:15 .
drwxr-xr-x 1 root root 4096 Feb  2 12:16 ..
drwxr-xr-x 4 root root 4096 Jan 31 17:14 .config
drwxr-xr-x 1 root root 4096 Jan 31 17:15 sample_data
/content


In [0]:
# Let's check if a GPU accelerator card is attached in our machine:
!ls -l /dev/nv*

crw-rw-rw- 1 root root 195,   0 Feb  2 11:37 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Feb  2 11:37 /dev/nvidiactl
crw-rw-rw- 1 root root 248,   0 Feb  2 11:37 /dev/nvidia-uvm
crw-rw-rw- 1 root root 248,   1 Feb  2 11:37 /dev/nvidia-uvm-tools


In [0]:
# This works too, GPU count and name
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-95dc9ff4-cd65-3c55-a686-4764167feab4)


### Use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'

In [0]:
! nvidia-smi

Sat Feb  2 11:57:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


In [0]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Thread(s) per core:  2


In [0]:
#memory that we can use
!cat /proc/meminfo | grep 'MemAvailable'

MemAvailable:   12482952 kB


In [0]:
#hard disk that we can use
!df -h / | awk '{print $4}'

Avail
319G


In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [0]:
# This magic will create a new CU file
# To write a CUDA C program, we need to:

#Create a source code fi le with the special fi le name extension of .cu.
#Compile the program using the CUDA nvcc compiler.
#Run the executable file from the command line, which contains the kernel code executable on the GPU.

%%file version.cu
#include <thrust/version.h>
#include <iostream>

int main(void)
{
  int major = THRUST_MAJOR_VERSION;
  int minor = THRUST_MINOR_VERSION;

  std::cout << "Thrust v" << major << "." << minor << std::endl;

  return 0;
}

Overwriting version.cu


In [0]:
# nvcc is the CUDA compiler 
!nvcc version.cu -o version
!./version

Thrust v1.9


###  Memory footprint support libraries/code

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 12.8 GB  I Proc size: 142.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
import sys
sys.version


'3.6.7 (default, Oct 22 2018, 11:32:17) \n[GCC 8.2.0]'

In [0]:
# !pip3 install torch==0.4
# !pip3 install torchvision

!pip3 install 'torch==0.4.0'
!pip3 install 'torchvision==0.2.1'
!pip3 install --no-cache-dir -I 'pillow==5.1.0'

# Restart Kernel
# This workaround is needed to properly upgrade PIL on Google Colab.
import os
#os._exit(0)

    100% |████████████████████████████████| 2.0MB 23.8MB/s 
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.


## Let's print the versions

In [0]:
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

use_cuda = torch.cuda.is_available()
# use_cuda = False

print("USE CUDA=" + str (use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

__Python VERSION: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
__pyTorch VERSION: 0.4.0
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Tue_Jun_12_23:07:04_CDT_2018
Cuda compilation tools, release 9.2, V9.2.148
__CUDNN VERSION: 7102
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0
USE CUDA=True
